In [1]:
from data.Algorithem.global_repository import*
game_status_repository.now_turn_player=1
import socket,pickle
class Server_Client():
    def __init__(self,name="HOST"):
        #포트 조회
        self.name = name
        # self.server_socket = self.start_server()
        self.player = {}
        self.play_state = "greenroom"
        # self.start_recieved()
        self.maximum_count = 3
        pass

    def start_server(self,host='0.0.0.0', port=12345):
        server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        server_socket.bind((host, port))
        server_socket.listen(5)
        print(f'Server listening on {host}:{port}') 
        return server_socket

    def start_recived(self):
        global game_status_repository,round_status_repository,player_status_repository
        self.server_socket = self.start_server()
        while len(self.player) <self.maximum_count and self.play_state == "greenroom":
            client_socket, addr = self.server_socket.accept()
            client_ip = addr[0]
            print(f'Connection from {client_ip}')
            data_ori = client_socket.recv(1024).decode()
            print(data_ori)
            if "|" in data_ori:
                data = data_ori.split("|")[0]
                player = data_ori.split("|")[1]
            else:data=""
            match data:
                case "ping":
                    client_socket.send(f'pong|{self.name}|{len(self.player)}'.encode())
                case "OPEN?":
                    client_socket.send(f'yes'.encode())
                case "JOIN":
                    if len(self.player) ==self.maximum_count :
                        client_socket.send(f'Full'.encode())
                    else:
                        # client_socket.send(f'yes'.encode())
                        print(f"{player} 플레이어가 입장했습니다.")
                        self.player[client_ip,player]=[client_socket,len(self.player)]
                        print(self.player)
                # case "start":
                #     client_socket.send(f'Full'.encode())
                case "exit":
                    del self.player[client_ip]
                    client_socket.send(f'yes'.encode())
                    print(self.player)
                    print(f"{player} 플레이어가 나갔습니다.")
                case _:
                    client_socket.send(f'error'.encode())
                    print("error")
        print("게임이 시작되었습니다.")
        self.play_state="playing"
        # while self.play_state == "playing":
        for i in range(len(self.player)):
            player_status_repository[i].name=list(self.player.keys())[i]
        
        for client_socket,player_num in self.player.values():            
            client_socket.send(pickle.dumps([game_status_repository,round_status_repository,player_status_repository]))

        while self.play_state == "playing":
            client_socket, addr = self.server_socket.accept()
            client_ip = addr[0]
            data_ori = client_socket.recv(9192)
            # print(f'Connection from {client_ip}')
            for i in self.player:
                if client_ip in i:
                    self.player[i][0]= client_socket
            if "wait|".encode() in data_ori :
                data = data_ori.decode().split("|")[0]
                player = data_ori.decode().split("|")[1]
                # self.player[client_ip,player][0] = client_socket
            else:
                print(data_ori)
                response = pickle.loads(data_ori)
                game_status_repository = response[0]
                round_status_repository = response[1]
                player_status_repository = response[2]
                for client,player_num in self.player.values():
                    # client_socket.send(data_ori)
                    client.send(pickle.dumps([game_status_repository,round_status_repository,player_status_repository]))
            # client_socket.send(data_ori)
            # client_socket.send(pick|e.dumps([game_status_repository,round_status_repository,player_status_repository]))
            if "exit".encode() in data_ori:
                self.play_state = ""
            print(game_status_repository.now_turn_player)
        self.server_socket.close()
        

In [2]:
a.server_socket.close()

NameError: name 'a' is not defined

In [3]:
a = Server_Client()
a.start_recived()

Server listening on 0.0.0.0:12345
Connection from 192.168.0.5
JOIN|김도훈
김도훈 플레이어가 입장했습니다.
{('192.168.0.5', '김도훈'): [<socket.socket fd=772, family=2, type=1, proto=0, laddr=('192.168.0.5', 12345), raddr=('192.168.0.5', 57661)>, 0]}
Connection from 192.168.0.5
OPEN?|김선후
Connection from 192.168.0.5
JOIN|김선후
김선후 플레이어가 입장했습니다.
{('192.168.0.5', '김도훈'): [<socket.socket fd=772, family=2, type=1, proto=0, laddr=('192.168.0.5', 12345), raddr=('192.168.0.5', 57661)>, 0], ('192.168.0.5', '김선후'): [<socket.socket fd=1936, family=2, type=1, proto=0, laddr=('192.168.0.5', 12345), raddr=('192.168.0.5', 57668)>, 1]}
Connection from 192.168.0.5
JOIN|김성훈
김성훈 플레이어가 입장했습니다.
{('192.168.0.5', '김도훈'): [<socket.socket fd=772, family=2, type=1, proto=0, laddr=('192.168.0.5', 12345), raddr=('192.168.0.5', 57661)>, 0], ('192.168.0.5', '김선후'): [<socket.socket fd=1936, family=2, type=1, proto=0, laddr=('192.168.0.5', 12345), raddr=('192.168.0.5', 57668)>, 1], ('192.168.0.5', '김성훈'): [<socket.socket fd=1952, family=2,

In [4]:
game_status_repository.now_turn_player

2

In [4]:
a.player

{('192.168.0.5',
  '김도훈'): [<socket.socket fd=1916, family=2, type=1, proto=0, laddr=('192.168.0.5', 12345), raddr=('192.168.0.5', 56594)>, 0],
 ('192.168.0.5',
  '김선후'): [<socket.socket fd=404, family=2, type=1, proto=0, laddr=('192.168.0.5', 12345), raddr=('192.168.0.5', 56596)>, 1]}

In [7]:
a.player.keys()

dict_keys([('192.168.0.5', '김도훈'), ('192.168.0.5', '김선후')])

In [9]:
for l in a.player:
    print(l)

('192.168.0.5', '김도훈')
('192.168.0.5', '김선후')
